# 10 - Gameplay and Decision Making

In [2]:
from constants import *
import numpy as np

In [2]:
# Both center of mass calculations only need to consider every other layer (the axis aligned ones)
# because the others are balanced regardless of which blocks are left
# In any axis aligned layer, blocks 1, 2, and 3 have centers at -1, 0, and 1 respectively if we place
# the origin and the z-axis at the center of the stack and use one block width as our unit distance
def compute_cms(stack, compute_x):
    # subtower i is the tower consisting layers i, i-1,..., n
    num_blocks_by_subtower = np.cumsum(np.sum(stack, axis = 1))[::-1]
    weight_dist_by_layer = np.zeros(stack.shape[0])
    # only consider even layers for x CoMs and odd layers for y CoMs
    start_layer = 0 if compute_x else 1
    # unnormalized CoM e.g. a layer of [1, 1, 0] will evaluate to -1 (instead of the CoM -1 / 2 = -0.5)
    weight_dist_by_layer[start_layer::2] = np.sum(stack[start_layer::2] * BLOCK_POSITIONS, axis = 1)
    weight_dist_by_subtower = np.cumsum(weight_dist_by_layer)[::-1]
    return weight_dist_by_subtower / num_blocks_by_subtower

def compute_x_cms(stack):
    return compute_cms(stack, compute_x = True)

def compute_y_cms(stack):
    return compute_cms(stack, compute_x = False)

In [7]:
# given a single layer of blocks (as a boolean array), determine whether the layer can support the tower
# above by checking if the center of mass (of the tower above) falls within the layer's support polygon
def within_support_polygon(cm, layer):
    if np.all(layer == 0):
        return False
    idx = np.where(layer)[0]
    return LEFT_POSITIONS[idx[0]] <= cm <= RIGHT_POSITIONS[idx[-1]]

# filter out moves that will cause the tower to topple when the block is removed
def validate_removals(stack, moves):
    x_cms, y_cms = compute_x_cms(stack), compute_y_cms(stack)
    blocks_above_layer = np.append(np.cumsum(np.sum(stack, axis = 1))[::-1], 0)[1:]

    output = []
    # ensure that the tower won't topple if the block is removed
    for move in moves:
        block_layer, block_index = move
        valid_for_layers_below = True
        # check all layers below removed block to ensure the center of mass won't shift too much
        for layer_below in range(block_layer):
            # check x_cm if layer below is even and y_cm if layer below is odd
            old_cm = x_cms[layer_below + 1] if (layer_below % 2) else y_cms[layer_below + 1]
            n = blocks_above_layer[layer_below]
            new_cm = old_cm * (n / (n - 1))
            # if the axes are not aligned, then the contribution of the block to
            # CM is 0, but we will need to adjust if the axes are aligned
            if (block_layer % 2) == (layer_below % 2):
                new_cm -= (block_index - 1) / (n - 1)
            # the center of mass will fall outside the support polygon, so this move is invalid
            if not within_support_polygon(new_cm, stack[layer_below]):
                valid_for_layers_below = False
                break
        # skip over invalid move
        if not valid_for_layers_below:
            continue

        # finally ensure that there's no rug pull
        above_cm = (x_cms if (block_layer % 2) else y_cms)[block_layer + 1]
        # create layer without the candidate block
        layer = stack[block_layer].copy()
        layer[block_index] = 0
        # move has passed all tests
        if within_support_polygon(above_cm, layer):
            output.append(move)
    return output

def find_valid_moves(stack):
    height = stack.shape[0]
    # can't move blocks from top layer
    possible_moves = [(i, j) for (i, j) in np.ndindex(*stack.shape) if (stack[i, j] and i != height - 1)]

    # ensure that simply removing the block won't topple the stack
    possible_moves = validate_removals(stack, possible_moves)

    # add block to top of tower
    num_blocks_top_layer = np.sum(stack[-1])
    # add new layer
    if num_blocks_top_layer == 3:
        stack = np.append(stack, np.eye(NUM_BLOCKS_PER_LAYER)[0])
    else:
        stack[-1, num_blocks_top_layer] = 1

    # also ensure that adding the block to the top won't topple the stack
    return validate_removals(stack, possible_moves)

In [ ]:
class Stack:
    def __init__(self, num_players):
        self.players = [Player(self) for _ in num_players]
        self.tower_height = NUM_LAYERS
        self.move_count = 0
        self.boolean_stack = np.ones((NUM_LAYERS, NUM_BLOCKS_PER_LAYER))

    @property
    def toppled(self):
        return False

    @property
    def valid_moves(self):
        return find_valid_moves(self.boolean_stack)


class Player:
    def __init__(self, stack):
        self.stack = stack
        self.stack_tightness = np.zeros((NUM_LAYERS, NUM_BLOCKS_PER_LAYER))

    # gradually push block at given index to measure how tight it is
    def test_block(block):
        pass

    # move block from given index to top of stack
    def move_block(block):
        pass

    def move(self):
        # prioritize moves that are closer to the top by reversing list
        possible_moves = self.stack.valid_moves[::-1]
        best_move, best_tightness = None, np.inf
        for move in possible_moves:
            # unknown block
            if self.stack_tightness[*move] == 0.:
                self.test_block(move)
            # good enough to move
            if self.stack_tightness[*move] < FRICTION_THRESHOLD:
                self.move_block(move)
                return
            # otherwise track the best available
            elif self.stack_tightness[*move] < best_tightness:
                best_move, best_tightness = move, self.stack_tightness[*move]
        # take the best available move if there were no loose blocks
        self.move_block(best_move)

In [ ]:
stack = Stack()
while not stack.toppled:
    for player in stack.players:
        player.move()
        if stack.toppled:
            break